## Homework 8 - part 1

## <em> VI, EL$_2$O, Generative Models, Multimodal Posteriors, and Gaussian Processes</em>
<br>
This notebook is arranged in cells. Texts are usually written in the markdown cells, and here you can use html tags (make it bold, italic, colored, etc). You can double click on this cell to see the formatting.<br>
<br>
The ellipsis (...) are provided where you are expected to write your solution but feel free to change the template (not over much) in case this style is not to your taste. <br>
<br>
<em>Hit "Shift-Enter" on a code cell to evaluate it.  Double click a Markdown cell to edit. </em><br>


***
### Link Okpy

In [ ]:
from client.api.notebook import Notebook
ok = Notebook('hw8_288.ok')
_ = ok.auth(inline = True)

### Imports

In [ ]:
import os
%pylab inline
import pickle
import corner
import time
PROJECT_PATH = os.path.abspath('../')
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_hub as hub
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow.contrib.distributions import softplus_inverse

***

#### Problem 1 - Image reconstruction from noisy and incomplete images

This exercise is based on https://arxiv.org/pdf/1910.10046.pdf:

Data reconstruction from corrupted data lies at the heart of Bayesian inverse problems. There are many ways in which data can be corrupted, ranging from missing data (or masking) to noise and blurring. In many cases the reconstructed data, e.g. an image, is high dimensional. Data reconstruction commonly faces two challenges: 1) For most data it is difficult to find a prior that optimally represents our knowledge of the uncorrupted data. Instead, priors are often chosen to impose certain regularity conditions (e.g. maximum entropy, smoothness). 2) While it is usually tractable to find the maximum of the posterior, i.e. the most probable underlying realization, a full uncertainty quantification of the reconstruction is usually prohibitively expensive.

We address these challenges with generative models, which provide a mapping from points in a typically lower dimensional latent space to points in the high dimensional data space. As an example of such a model we will be using a Variational AutoEncoder (VAE). 

VAEs are designed to model the distribution $p_{\phi}(\textbf{x})$ of high-dimensional input data, $\textbf{x}$, by introducing a mapping $p_{\phi}(\textbf{x}|\textbf{z})$ to a lower dimensional latent representation, $\textbf{z}$. The latent space variables are enforced to follow a given prior distribution, $p(\textbf{z})$, which is typically chosen to be a standard normal distribution.

Given a generative model, the posterior of the latent variables for a given data realization can be modeled with Bayes rule
$$ p_{\phi}(\textbf{z}|\textbf{x}) \propto p_{\phi}(\textbf{x}|\textbf{z})p(\textbf{z}) $$

This formulation allows to address both aforementioned problems: 1) The prior distribution, p(z), reflects the distribution of the training data. 2) The representation of the posterior in the lower dimensional latent space enables tractable posterior analysis. In particular it allows to examine and fit the posterior distribution and draw samples from it. The samples can then be visualized in data space by forward modeling with the generative model.

In this exercise, consider examples of data corruption on the MNIST dataset. First, load the MNIST dataset.

In [ ]:
# functions to load mnist dataset
import gzip, zipfile, tarfile
import os, shutil, re, string, urllib, fnmatch
import pickle as pkl

def _download_mnist_realval(dataset):
    """
    Download the MNIST dataset if it is not present.
    :return: The train, test and validation set.
    """
    origin = (
        'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
    )
    print('Downloading data from %s' % origin)
    urllib.request.urlretrieve(origin, dataset)

def _get_datafolder_path():
    full_path = os.path.abspath('.')
    path = full_path +'/data'
    return path

def load_mnist_realval(
        dataset=_get_datafolder_path()+'/mnist_real/mnist.pkl.gz'):
    '''
    Loads the real valued MNIST dataset
    :param dataset: path to dataset file
    :return: None
    '''
    if not os.path.isfile(dataset):
        datasetfolder = os.path.dirname(dataset)
        if not os.path.exists(datasetfolder):
            os.makedirs(datasetfolder)
        _download_mnist_realval(dataset)

    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = pkl.load(f, encoding='latin1')
    f.close()
    x_train, targets_train = train_set[0], train_set[1]
    x_valid, targets_valid = valid_set[0], valid_set[1]
    x_test, targets_test = test_set[0], test_set[1]
    return x_train, targets_train, x_valid, targets_valid, x_test, targets_test
  
x_train, targets_train, x_valid, targets_valid, x_test, targets_test = load_mnist_realval()

Then, let's create a corrupted image. Here, we mask half of the image and add noise with to the other half.

In [ ]:
data_dim    = 28*28 # data dimensionality
data_size   = 1     # number of corrupted input data, always 1
sigma_n     = 0.1   # noise level used in the likelihood, determined during VAE training or measured
hidden_size = 10    # latent space dimensionality of the VAE
sample_size = 512   # number of samples drawn in each step of the ELBO minimization (for stochastic VI only)
n_channels  = 1     # mnist is b/w -> 1 channel
seed        = 777   # random seed

### choose one of the following to run different examples

# settings for reconstruction with noise and mask
corr_type   = 'noise+mask'
num_mnist   = 6
label       = 'masknoise05'
noise_level = 0.5
num_comp    = 3

PROJECT_PATH = os.path.abspath('../')
plot_path        = os.path.join(PROJECT_PATH,'plots/')

# Saving plots
plot_path = os.path.join(plot_path,'%s/'%label)

if not os.path.isdir(plot_path):
    os.makedirs(plot_path)
    
def plot_image(image, save=True, directory='./plots/',filename='plotted_image', title='image',vmin=None,vmax=None, mask=None):
    """
    plots and saves a single image of mnist format
    """

    if np.any(mask==None):
        mask=np.ones_like(image)
    mask = np.reshape(mask,(28,28))
    plt.figure()
    #plt.title(title)
    plt.imshow((image).reshape((28,28))*mask,cmap='gray',vmin=vmin, vmax=vmax)
    plt.axis('off')
    #plt.colorbar()
    if save: 
        plt.savefig(directory+filename+'.pdf',bbox_inches='tight')
    plt.show()

    return True

def get_custom_noise(shape, signal_dependent=False, signal =None, sigma_low=0.07, sigma_high=0.22, threshold=0.02 ):
    """
    adds the model error and the noise level in the data to obtain the total sigma that is used in the likelihood (option to make the reconstruction error signal dependent is not used) 
    """
    sigma = np.ones(shape)*sigma_n

    if signal_dependent: 
        for ii in range(data_size):
            sigma[ii][np.where(signal[ii]<=threshold)]= sigma_low
            sigma[ii][np.where(signal[ii]>threshold)]= sigma_high

    data_noise = np.ones_like(sigma)*noise_level

    sigma = np.sqrt(sigma**2+data_noise**2)

    return sigma
  

def make_corrupted_data(x_true, corr_type='mask'):
    """
    creates the corrupted image according to chosen corruption type
    """

    mask = np.ones((28,28))

    if corr_type=='mask':

        minx = 10
        maxx = 24

        mask[0:28,minx:maxx]=0.
        mask = mask.reshape((28*28))

        corr_data = x_true*[mask]

    elif corr_type=='sparse mask':

        mask    = np.ones(data_dim, dtype=int)
        percent = 95
        np.random.seed(seed+2)
        indices = np.random.choice(np.arange(data_dim), replace=False,size=int(percent/100.*data_dim))
        print('precentage masked:', len(indices)/data_dim)
        mask[indices] =0 

        corr_data = x_true*[mask]

    elif corr_type=='noise':

        np.random.seed(seed+2)
        noise = np.random.randn(data_dim*data_size)*noise_level

        corr_data = x_true+noise

    elif corr_type=='noise+mask':

        np.random.seed(seed+2)
        noise = np.random.randn(data_dim*data_size)*noise_level

        minx = 14
        maxx = 28

        mask[0:28,minx:maxx]=0.
        mask = mask.reshape((28*28))

        corr_data = x_true+noise
        corr_data = corr_data*[mask]

    elif corr_type=='none':

        corr_data = x_true

    corr_data = np.expand_dims(corr_data,-1)

    mask = mask.flatten()

    return corr_data, mask

Using a pre-defined function "plot_image," you can easily plot an image of MNIST datasets. Below we plot corrupted data and the underlying truth. Note that plots are saved in HW8/plots/ in your datahub directory.

In [ ]:
# create underlying truth
truth = x_test[num_mnist:num_mnist+data_size]
# create corrupted data
data, custom_mask = make_corrupted_data(truth, corr_type=corr_type)
# noise
noise = get_custom_noise(data.shape, signal_dependent=False, signal=truth)

# Make plot
print('Truth:')
plot_image(truth, directory=plot_path, filename='truth_%s'%label, title='truth')
print('Data:')
plot_image(data, directory=plot_path, filename='input_data_%s'%label, title='data')

We already trained a VAE on the uncorrupted training set with both encoder, $f_{\psi}$, and decoder, $g_{\phi}$, parameterized as a sequence of 4 fully-connected ResNet blocks, each block containing 2 fully-connected layers with size 512 and LeakyRelu activation. The encoder reduces the dimensionality to 10. We minimize the loss using ADAM with default parameters, a batch size of 1024, and a decreasing learning rate starting at 0.001. We only give you already trained generative models because creating such VAEs is beyond the scope of this course.

To ensure that the prior is well described by a unit variance Gaussian, we augment the forward model by a normalizing flow, using RealNVP. The normalizing flow is trained to map the latent space distribution of the VAE to a standard normal distribution.

A more detailed procedure is described below.
![alt text](pic1.png "Title")
We train a vanilla VAE with Gaussian likelihood on uncorrupted training data.
![alt text](pic2.png "Title")
Then, in a second step a bijective normalizing flow is fitted to map the distribution of the encoded training data to a normal distribution.

In [ ]:
# pathes to trained generative models
generator_path   = os.path.join(PROJECT_PATH,'modules/decoder1/decoder')
encoder_path     = os.path.join(PROJECT_PATH,'modules/encoder1/encoder')
nvp_func_path    = os.path.join(PROJECT_PATH,'modules/nvp1/')
minima_path      = os.path.join(PROJECT_PATH,'minima/')

With the help of VAE and normalizing flow, we can do the posterior analysis in the latent space. We choose the dimension of the latent space to be 10 (set by "hidden_size" earlier). First, write a function for the log posterior:

In [ ]:
def fwd_pass(generator,nvp,z,mask):
    """
    a pass through the forward model (nvp+VAE generator)
    """ 

    fwd_z           = nvp({'z_sample':np.zeros((1,hidden_size)),'sample_size':1, 'u_sample':z},as_dict=True)['fwd_pass']

    gen_z           = tf.boolean_mask(tf.reshape(generator(fwd_z),[data_size,data_dim,n_channels]),mask, axis=1)

    return gen_z


def get_likelihood(generator,nvp,z,sigma,mask):
    """
    compute the likelihood for a given latent space vector
    """
    gen_z           = fwd_pass(generator,nvp,z,mask)

    sigma           = tf.boolean_mask(sigma,mask, axis=1)

    likelihood      = tfd.Independent(tfd.MultivariateNormalDiag(loc=gen_z,scale_diag=sigma))

    return likelihood

def get_prior():
    """
    return the Gaussian prior
    """

    return tfd.MultivariateNormalDiag(tf.zeros([data_size,hidden_size]), scale_identity_multiplier=1.0, name ='prior')

def get_log_posterior(z,x,generator,nvp,sigma,mask, beta):
    """
    returns the negative log posterior (or rather joint dsitribution) - log(p(x,z)), where x is the corrupted input image 
    """

    likelihood      = get_likelihood(generator,nvp,z,sigma,mask)

    prior           = get_prior()

    masked_x        = tf.boolean_mask(x,mask, axis=1)

    log_posterior   = prior.log_prob(z)+likelihood.log_prob(masked_x)*beta

    return log_posterior


def get_recon(generator,nvp, z,sigma,mask):
    """
    forward models a given point in latent space to data space
    """

    prob = get_likelihood(generator,nvp, z,sigma,mask)

    recon= prob.mean()

    return recon

def get_hessian(func, z):
    """
    computes the hessian of function func at point z
    """

    hess             = tf.hessians(func,z)
    hess             = tf.gather(hess, 0)

    return(tf.reduce_sum(hess, axis = 2 ))


def get_GN_hessian(generator,nvp,z,mask,sigma):
    '''
    computes the Gauss Newton approximation to the Hessian
    '''

    gen_z            = fwd_pass(generator,nvp,z,mask)

    sigma            = tf.boolean_mask(sigma,mask, axis=1)

    grad_g           = tf.gather(tf.gradients(gen_z/(sigma),z),0)

    grad_g2          = tf.einsum('ij,ik->ijk',grad_g,grad_g)

    one              = tf.linalg.eye(hidden_size, batch_shape=[data_size],dtype=tf.float32)

    hess_GN          = one+grad_g2

    return hess_GN
  
def compute_covariance(hessian):
    """
    computes the covariance by inverting the hessian (symmetrization for numerical stability)
    """

    cov = tf.linalg.inv(hessian)

    cov = (cov+tf.linalg.transpose(cov))*0.5

    return cov

def get_random_start_values(num, my_sess):
    """
    returns random starting values drawn from the prior
    """
    result=[]
    for ii in range(num):
        result.append(my_sess.run(get_prior().sample()))
    return result

Next, we define variables and functions needed to perform the posterior analysis.

In [ ]:
tf.reset_default_graph()

#### Variables ####
#noise level 
sigma_corr  = tf.placeholder_with_default(np.ones([data_size,data_dim,n_channels], dtype='float32')*sigma_n,shape=[data_size,data_dim,n_channels])
#mask
mask        = tf.placeholder_with_default(np.ones([data_dim], dtype='float32'),shape=[data_dim])
#corrupted input data
input_data  = tf.placeholder(shape=[data_size,data_dim,n_channels], dtype=tf.float32)
#temperature used for annealed minimzation
inverse_T   = tf.placeholder_with_default(1., shape=[])
#learning rate
lr          = tf.placeholder_with_default(0.001,shape=[])

#### Modules ####
generator   = hub.Module(generator_path, trainable=False)
nvp_funcs   = hub.Module(nvp_func_path, trainable=False)

In [ ]:
#### for Maximum a Posteriori estimate ####
MAP_ini     = tf.placeholder_with_default(tf.zeros([data_size,hidden_size]),shape=[data_size,hidden_size])
MAP         = tf.Variable(MAP_ini)
MAP_reset   = tf.stop_gradient(MAP.assign(MAP_ini))

nlPost_MAP  = get_log_posterior(MAP, input_data, generator,nvp_funcs, sigma_corr,mask, inverse_T)
loss_MAP    = -tf.reduce_mean(nlPost_MAP)

optimizer   = tf.train.AdamOptimizer(learning_rate=lr)
opt_op_MAP  = optimizer.minimize(loss_MAP, var_list=[MAP])

recon_MAP   = get_recon(generator,nvp_funcs, MAP,sigma_corr,mask)

In [ ]:
##### for Laplace approximation around a point ####
hessian     = get_hessian(-nlPost_MAP,MAP)
GN_hessian  = get_GN_hessian(generator,nvp_funcs,MAP,mask,sigma_corr)
ini_val  = np.ones((data_size,(hidden_size *(hidden_size +1)) // 2),dtype=np.float32)
with tf.variable_scope("Laplace_Posterior",reuse=tf.AUTO_REUSE):
    mu_new      = tf.Variable(np.ones((data_size,hidden_size),dtype=np.float32), dtype=np.float32)
    sigma_new_t = ini_val
    sigma_new_t2= tf.Variable(tfd.matrix_diag_transform(tfd.fill_triangular(sigma_new_t), transform=tf.nn.softplus),dtype=tf.float32)

approx_posterior_laplace = tfd.MultivariateNormalTriL(loc=mu_new,scale_tril=sigma_new_t2)

update_mu          = mu_new.assign(MAP)
covariance         = compute_covariance(hessian)
variance           = tf.linalg.diag_part(covariance)[0]
update_TriL        = sigma_new_t2.assign(tf.linalg.cholesky(covariance))

posterior_sample   = approx_posterior_laplace.sample()

recon              = get_recon(generator,nvp_funcs, posterior_sample ,sigma_corr,mask)

In [ ]:
#for ELBO

########## Syochastic VI estimates by minimizing the elbo #############

def get_neg_elbo(x,approx_posterior, generator, nvp, sigma, mask):
    """
    computes the negative evidence lower bound (ELBO)
    """

    prior          = get_prior()
    kl_divergence  = tfd.kl_divergence(approx_posterior, prior) 

    z_sample       = tf.reshape(approx_posterior.sample(sample_size),[-1,hidden_size])

    fwd_z          = nvp({'z_sample':np.zeros((1,hidden_size)),'sample_size':1, 'u_sample':z_sample},as_dict=True)['fwd_pass']
    gen_z          = tf.boolean_mask(tf.reshape(generator(fwd_z),[sample_size,data_size,data_dim,n_channels]),mask, axis=2)
    sig            = tf.boolean_mask(sigma,mask, axis=1)

    likelihood     = tfd.Independent(tfd.MultivariateNormalDiag(loc=gen_z,scale_diag=sig))

    masked_x       = tf.boolean_mask(x, mask, axis=1)
    like_prob      = likelihood.log_prob(tf.expand_dims(masked_x,0))

    elbo           = tf.reduce_mean(like_prob)- kl_divergence

    return -elbo

def minimize_neg_elbo(x,custom_mask,noise,my_sess,full_rank=False):
    """
    function to run a ELBO minimzation for stochastic VI.
    """


    if full_rank:
        sets = zip([1e-3,1e-4,1e-6],[2000,500,300])
    else:
        sets = zip([1e-3,1e-4,1e-5],[2000,500,300])

    elbo_loss = []
    start = time.time()
    for lrate, numiter in sets:
        print('lrate', lrate)
        for jj in range(numiter):
            if full_rank:
                _, ll = my_sess.run([opt_op_elbo_fr,neg_elbo_fr],feed_dict={input_data: x, mask:custom_mask, sigma_corr:noise, lr: lrate})
            else:
                _, ll = my_sess.run([opt_op_elbo,neg_elbo],feed_dict={input_data: x, mask:custom_mask, sigma_corr:noise, lr: lrate})
            elbo_loss.append(ll)
            if jj%1000==0:
                print('iter', jj, 'loss', ll)
                #if full_rank:
                #  print(my_sess.run([mu_vi,post_vi.covariance()],feed_dict={input_data: x, mask:custom_mask, sigma_corr:noise, lr: lrate}))
    end= time.time()
    print('time taken',end-start)
    loss    = ll
    plt.figure()
    plt.plot(elbo_loss)
    plt.ylabel('loss')
    plt.xlabel('iteration')
    plt.show()

    return True
  
def add_small_offset(x):
    """offset added to the diagonal of the covariance, needed to make SVI stable"""
    return tf.add(x,1e-8)

#### mean field VI ####
with tf.variable_scope("posterior_elbo_mean",reuse=tf.AUTO_REUSE):
    mu_elbo      = tf.Variable(np.zeros((data_size,hidden_size)), dtype=np.float32)
    sigma_elbo   = tf.Variable(np.ones((data_size,hidden_size)), dtype=np.float32)

update_mu_elbo = tf.stop_gradient(mu_elbo.assign(MAP))

approx_posterior_elbo = tfd.MultivariateNormalDiag(loc=mu_elbo,scale_diag=sigma_elbo, name='approxposterior_elbo')

neg_elbo       = get_neg_elbo(input_data, approx_posterior_elbo, generator, nvp_funcs, sigma_corr, mask)
opt_op_elbo    = optimizer.minimize(neg_elbo,var_list=[mu_elbo,sigma_elbo])

nlPost_elbo    = -tf.reduce_mean(get_log_posterior(mu_elbo, input_data, generator,nvp_funcs, sigma_corr,mask, 1))
#######################


### full rank VI ###
ini_val1     = np.zeros((data_size,(hidden_size *(hidden_size +1)) // 2),dtype=np.float32)
with tf.variable_scope("posterior_elbo_fullrank",reuse=tf.AUTO_REUSE):
    mu_vi      = tf.Variable(np.ones((data_size,hidden_size)), dtype=np.float32)
    sigma_vi   = tf.Variable(ini_val1)
sigma_vit    = tfd.matrix_diag_transform(tfd.fill_triangular(sigma_vi), transform=tf.nn.softplus)
sigma_vit    = tfd.matrix_diag_transform(sigma_vit, transform=add_small_offset)
post_vi      = tfd.MultivariateNormalTriL(loc=mu_vi,scale_tril=sigma_vit)

neg_elbo_fr    = get_neg_elbo(input_data, post_vi, generator, nvp_funcs, sigma_corr, mask)
opt_op_elbo_fr = optimizer.minimize(neg_elbo_fr,var_list=[mu_vi,sigma_vi])

hess_fr        = tf.linalg.inv(post_vi.covariance())
nlPost_vi      = -tf.reduce_mean(get_log_posterior(mu_vi, input_data, generator,nvp_funcs, sigma_corr,mask, 1))

recon_fr       = get_recon(generator,nvp_funcs, mu_vi ,sigma_corr,mask)
recon_elbo     = get_recon(generator,nvp_funcs, mu_elbo ,sigma_corr,mask)
####################

In [ ]:
##### for Gaussian mixture model fit around minima #####
ini_val2    = np.ones((data_size,num_comp,(hidden_size *(hidden_size +1)) // 2),dtype=np.float32)
with tf.variable_scope("corrupted/gmm",reuse=tf.AUTO_REUSE):
    mu_gmm      = tf.Variable(np.ones((data_size,num_comp,hidden_size)), dtype=np.float32)
    sigma_gmm   = tf.Variable(tfd.fill_triangular(ini_val2))
    w_gmm       = tf.Variable(np.ones((num_comp))/num_comp, dtype=np.float32)

sigma_gmmt    = tfd.matrix_diag_transform(sigma_gmm, transform=tf.nn.softplus)
w_positive    = tf.math.softplus(w_gmm)
w_rescaled    = tf.squeeze(w_positive/tf.reduce_sum(w_positive))

gmm           = tfd.MixtureSameFamily(mixture_distribution=tfd.Categorical(probs=w_rescaled),components_distribution=tfd.MultivariateNormalTriL(loc=mu_gmm,scale_tril=sigma_gmmt))

mu_ini        = tf.placeholder_with_default(tf.zeros([data_size,num_comp,hidden_size]),shape=[data_size,num_comp,hidden_size])
sigma_ini     = tf.placeholder_with_default(tf.ones([data_size,num_comp,hidden_size, hidden_size]),shape=[data_size,num_comp,hidden_size, hidden_size])
w_ini         = tf.placeholder_with_default(tf.ones([num_comp])/num_comp,shape=[num_comp])

update_w      = tf.stop_gradient(w_gmm.assign(softplus_inverse(w_ini)))
update_mugmm  = tf.stop_gradient(mu_gmm.assign(mu_ini))
update_TriLgmm= tf.stop_gradient(sigma_gmm.assign(tfd.matrix_diag_transform(sigma_ini, transform=softplus_inverse)))

gmm_sample    = gmm.sample()
gmm_recon     = get_recon(generator,nvp_funcs, gmm_sample ,sigma_corr,mask)
####################################################


We will run TensorFlow to perform the posterior analysis. Running TensorFlow requires opening up a `Session` which we abbreviate as `sess` for short. All operations are performed in this session by calling the `run` method. First, we initialize the global variables in TensorFlow's computational graph by running the `global_variables_initializer`. 

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

tf.random.set_random_seed(seed)

We use optimization to find all local minima of the negative log posterior. We start from 15 different random positions drawn from the prior and use the ADAM optimizer to descend to the local minima. We stop our search once we find that the minimization procedure repeatedly converges to the same minima, typically after ~10 minimizations.

In [ ]:
inits = get_random_start_values(15, sess)

Let us try 3 different learning rate values: 1e-1 with 1100 iterations, 1e-2 with 400 iterations, 1e-3 with 100 iterations.

In [ ]:
lrate_run1 = np.array([1e-1,1e-2,1e-3])
numiter_run1 = np.array([1100,400,100])

Define a function for minimizing a posterior loss. 

In [ ]:
def minimize_posterior(initial_value, x, custom_mask, noise, my_sess, annealing =True):
    """
    function to run several minimizations of the posterior
    """

    ini = np.reshape(initial_value,[data_size,hidden_size])

    my_sess.run(MAP_reset,feed_dict={input_data: x, MAP_ini:ini, mask:custom_mask,sigma_corr:noise})

    pos_def = False

    start = time.time()
    posterior_loss = []
    for lrate, numiter in zip(lrate_run1, numiter_run1):
        for jj in range(numiter):
            if annealing and lrate==1e-1:
                inv_T= np.round(0.5*np.exp(-(1.-jj/numiter)),decimals=1)
            else:
                inv_T= 1.
            _, ll = my_sess.run([opt_op_MAP,loss_MAP],feed_dict={input_data: x, mask:custom_mask, sigma_corr:noise, lr: lrate, inverse_T:inv_T})
            posterior_loss.append(ll)
    end = time.time()
    z_value = my_sess.run(MAP,feed_dict={input_data: x, mask:custom_mask, sigma_corr:noise})

    eig     = my_sess.run(tf.linalg.eigvalsh(hessian),feed_dict={input_data: x, mask:custom_mask,sigma_corr:noise})
    if np.all(eig>0.):
        pos_def = True

    loss    = ll

    return z_value, loss, pos_def
  

Run 15 differnt minimizations. See the below code: for each minimization run, we save the latent variable values to `minima`, and the posterior loss values are saved to `min_loss`.

In [ ]:
minima  =[]
min_loss=[]
min_var =[]
recons  =[]
for jj,init in enumerate(inits):
    print('progress in %', jj/len(inits)*100)
    min_z, min_l, pos_def    = minimize_posterior(init, data,custom_mask,noise,sess)
    rec                      = sess.run(recon_MAP, feed_dict={sigma_corr:noise})
    var                      = sess.run(variance, feed_dict={input_data: data,mask:custom_mask,sigma_corr:noise})
    print('reconstruction run %d (loss %.1f)'%(jj+1, min_l))
    plot_image(rec, directory=plot_path, filename='recon_%s_minimum%d'%(label,jj), title='reconstruction with loss %.1f'%min_l)
    #If Hessian if positive definite
    if pos_def:
        minima.append(min_z)
        min_loss.append(min_l)
        min_var.append(var)
        recons.append(rec)

Below we order the minimization results in the ascending order so that `minima[0]` corresponds to the latent variable values at the deepest minimum of the negative log posterior, which is the MAP solution.

In [ ]:
order    = np.argsort(min_loss)
min_loss = np.asarray(min_loss)[order]
minima   = np.asarray(minima)[order]
min_var  = np.asarray(min_var)[order]

Save your minimization results.

In [ ]:
pickle.dump([minima, min_loss, min_var,recons],open(minima_path+'minima_%s.pkl'%label,'wb'))

So even if your kernel died due to the memory overflow, you can retrieve your results by running the below cell. (Uncomment the below cell and run)

In [ ]:
# minima, min_loss, min_var, recons = pickle.load(open(minima_path+'minima_%s.pkl'%label,'rb'))

Plot the minimization results.

In [ ]:
def plot_minima(minima, losses, var):
    """
    plot the minima for comparison with their width estimated from the variance at these points -> helps decide whether minima are separate
    """
    plt.figure()
    plt.title('Minimization result')
    plt.plot(np.arange(len(losses)),losses,ls='',marker='o')
    plt.xlabel('# iteration')
    plt.ylabel('loss')
    plt.savefig(plot_path+'minimzation_results_%s.png'%(label),bbox_inches='tight')
    plt.show()

    colors = matplotlib.colors.Normalize(vmin=min(losses), vmax=max(losses))
    cmap   = matplotlib.cm.get_cmap('Spectral')

    var = np.squeeze(var)
    plt.figure()
    plt.title('value of hidden variables at minima')
    for ii in range(len(minima)):

        yerr_= np.sqrt(var[ii])

        plt.errorbar(np.arange(hidden_size),np.squeeze(minima)[ii], marker='o',ls='', c=cmap(colors(losses[ii])), mew=0, yerr=yerr_, label ='%d'%losses[ii])
    plt.legend(ncol=4, loc=(1.01,0))
    plt.xlabel('# hidden variable')
    plt.ylabel('value')
    plt.savefig(plot_path+'hidden_values_at_minima_%s.png'%(label),bbox_inches='tight')
    plt.show()

In [ ]:
plot_minima(minima, min_loss, min_var)

Finally, set MAP value to the lowest minimum (minima[0])

In [ ]:
lowest_minimum = sess.run(MAP_reset, feed_dict={MAP_ini:minima[0]})

Plot the reconstruction results. 

In [ ]:
print('reconstruction from lowest minimum (MAP solution)')
rec     = sess.run(recon_MAP, feed_dict={sigma_corr:noise})
plot_image(rec, directory=plot_path, filename='lowest_minimum_%s'%(label), title='reconstruction', vmin=0, vmax=1)

Next, let us try stochastic VI (SVI) as a posterior fitting procedure. We use a pre-defined function (see above) 'minimize_neg_elbo' to stochastically minimize the evidence lower bound (ELBO) usig ADAM with decreasing learning rate (the default setting is running with the following learning rate values: 1e-3 with 2000 iterations, 1e-4 with 500 iterations, 1e-5 with 300 iterations - if you have enough memory, you can try more iterations). Here we try both mean field SVI (fitting Gaussians with diagonal) and full rank SVI (full rank covariance).

<span style="color:red"> <i> NOTE: If the kernel dies due to the memory overflow, you can simply re-load optimization results you saved and start from there.  </i></span> <br>

In [ ]:
sess.run(update_mu_elbo, feed_dict={input_data: data,mask:custom_mask,sigma_corr:noise})
print('starting mean field stochastic VI on the posterior')
minimize_neg_elbo(data,custom_mask,noise,sess)
print('starting full rank stochastic VI on the posterior')
minimize_neg_elbo(data,custom_mask,noise,sess, full_rank=True)

Plot the reconstructed images below:

In [ ]:
print('full rank SVI reconstruction')
rec     = sess.run(recon_fr, feed_dict={sigma_corr:noise})
plot_image(rec, directory=plot_path, filename='full_rank_VI_%s'%(label), title='reconstruction', vmin=0, vmax=1)
print('mean field SVI reconstruction')
rec     = sess.run(recon_elbo, feed_dict={sigma_corr:noise})
plot_image(rec, directory=plot_path, filename='mean_field_VI_%s'%(label), title='reconstruction', vmin=0, vmax=1)


You can observe that full rank SVI results do not look good. That is because we only used a small number of iterations due to the memory limit, suggesting that full rank VI converges to the solution slowly.

Now, we can try the EL$_2$O procedure (See slide 33, Lecture 9) - we find the minimum and fit a full rank Gaussian with the Laplace approximation.

In [ ]:
def get_laplace_sample(num,map_value,x,mymask,noise,my_sess):
    """
    samples from the Laplace approximation around the deepest minimum
    """

    my_sess.run(MAP_reset,feed_dict={MAP_ini:map_value})
    my_sess.run(update_mu)
    my_sess.run(update_TriL,feed_dict={input_data: x, mask: mymask, sigma_corr:noise})

    samples=[]
    for ii in range(num):
        my_sess.run(posterior_sample,feed_dict={input_data: x, sigma_corr:noise})
        samples.append(my_sess.run(recon,feed_dict={input_data: x, sigma_corr:noise}))

    samples=np.asarray(samples)
    return samples

def get_gmm_sample(num,x,mymask,noise,my_sess):
    """
    samples from the Gaussian mixture posterior fit
    """

    samples=[]
    for ii in range(num):
        samples.append(my_sess.run(gmm_recon,feed_dict={input_data: x, sigma_corr:noise}))

    samples=np.asarray(samples)
    return samples

def plot_samples(samples, mask, title='samples', filename='samples'):
    """
    plots a compilation of samples
    """
    plt.figure()
    plt.title(title)
    for i in range(min(len(samples),16)):
        subplot(4,4,i+1)
        imshow(np.reshape(samples[i,:],(28,28)),vmin=-0.2,vmax=1.2, cmap='gray')
        axis('off')
    plt.savefig(plot_path+filename+'.png',bbox_inches='tight')
    plt.show()

    if corr_type in ['mask', 'sparse mask', 'noise+mask']:
        plt.figure()
        plt.title('masked'+title)
        for i in range(min(len(samples),16)):
            subplot(4,4,i+1)
            imshow(np.reshape(samples[i,0,:,0]*mask,(28,28)),vmin=-0.2,vmax=1.2, cmap='gray')
            axis('off')     
        plt.savefig(plot_path+filename+'masked.png',bbox_inches='tight')
        plt.show()
        
def get_chi2(sigma,data,mean,masking=True, mask=None,threshold=0.02):
    """
    computes chi2 between data and mean
    """

    if masking:
        mask = np.reshape(mask,data.shape)
        data = data[np.where(mask==1)]
        mean = mean[np.where(mask==1)]
        sigma= sigma[np.where(mask==1)]


    low = min(sigma.flatten())
    high= max(sigma.flatten())

    chi2_tot = np.sum((data-mean)**2/sigma**2)
    dof_tot  = len(np.squeeze(data))

    if corr_type not in ['noise','noise+mask']:
        chi2_low = np.sum((data[np.where(data<=threshold)]-mean[np.where(data<=threshold)])**2/sigma[np.where(data<=threshold)]**2)
        dof_low  = len(np.squeeze(data[np.where(data<=threshold)]))
        chi2_high= np.sum((data[np.where(data>threshold)]-mean[np.where(data>threshold)])**2/sigma[np.where(data>threshold)]**2)
        dof_high = len(np.squeeze(data[np.where(data>threshold)]))
    else:
        chi2_low = None
        dof_low  = None
        chi2_high= None
        dof_high = None

    return chi2_tot, dof_tot, chi2_low, dof_low, chi2_high, dof_high, masking

def probe_posterior(minimum, x, noise, mymask, my_sess, filename=label):
    """
    probes the real posterior along each latent space direction around the minimum and compares to posterior fits
    """

    _ = my_sess.run(MAP_reset,feed_dict={input_data: x, MAP_ini:minimum, sigma_corr:noise})
    _ = my_sess.run(update_mu,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    _ = my_sess.run(update_TriL,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})

    exact_hessian = sess.run(hessian,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    approx_hessian= sess.run(GN_hessian,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    ll0 = sess.run(loss_MAP,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})

    mean_vi, sig_vi, ll0_vi = sess.run([mu_elbo,sigma_elbo, nlPost_elbo],feed_dict={input_data: x, mask:mymask, sigma_corr:noise})


    mean_fr, h_fr, ll0_fr   = sess.run([mu_vi,hess_fr, nlPost_vi],feed_dict={input_data: x, mask:mymask, sigma_corr:noise})


    print(mean_vi)
    print(minimum)

    print(ll0,ll0_vi,ll0_fr)

    plt.figure(figsize=(4*3,3))


    jj =0
    for nn in [5,6,7]:#np.arange(hidden_size):
        H    = exact_hessian[0,nn,nn]
        Hfr  = h_fr[0,nn,nn]
        H_vi = 1./sig_vi[0,nn]**2

        losses=[]

        print(H_vi,H)

        subplot(1,3,jj+1)

        title('latent space direction %d'%nn)

        Delta   = 0.5
        steps   = 2000
        steps_fine = 10000
        delta_z = np.zeros((steps,hidden_size))

        delta_z[:,nn] = (np.arange(steps)-steps//2)*Delta/steps
        new_ini       = delta_z+minimum
        new_ini_vi    = delta_z+mean_vi
        new_ini_fr    = delta_z+mean_fr

        full_span     = np.ones((steps_fine,hidden_size))*minimum
        full_span_min = min(min(new_ini_vi[:,nn]),min(new_ini[:,nn]),min(new_ini_fr[:,nn]))
        full_span_max = max(max(new_ini_vi[:,nn]),max(new_ini[:,nn]),max(new_ini_fr[:,nn]))
        full_span_    = np.linspace(full_span_min,full_span_max,steps_fine)
        full_span[:,nn]= full_span_

        for ii in range(steps_fine):
            _ = sess.run(MAP_reset,feed_dict={input_data: x, mask:mymask, MAP_ini:np.expand_dims(full_span[ii],axis=0), sigma_corr:noise})
            ll = sess.run(loss_MAP,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
            losses.append(ll)


        plt.plot(full_span[:,nn],losses,label='probed posterior', color='crimson',lw=2)
        plt.plot(new_ini[:,nn],ll0+H*delta_z[:,nn]**2,label='EL2O estimate (full rank)',lw=2)
        plt.plot(new_ini_vi[:,nn],ll0_vi+H_vi*delta_z[:,nn]**2,label='mean field SVI estimate',lw=2)
        plt.plot(new_ini_fr[:,nn],ll0_fr+Hfr*delta_z[:,nn]**2,label='full rank SVI estimate',lw=2)

        plt.xlabel('z',fontsize=16)
        plt.ylim(min(min(losses),ll0_fr),max(max(losses),ll0_fr+0.5))
        if jj==0:
            plt.ylabel('negative log posterior',fontsize=14)
            plt.legend(loc='upper right',fontsize=12,framealpha=0.9)
        jj+=1
    plt.tight_layout()




    plt.savefig(plot_path+'probing_posterior_%s.pdf'%(filename),bbox_inches='tight')
    plt.show()

    

def get_gmm_parameters(minima, x, noise, mymask, offset):
    """
    Gaussian mixture model (could be coded more elegantly with softmax function)
    """

    mu   =[]
    w    =[]
    sigma=[]
    for ii in range(num_comp):

        # do Laplace approximation around this minimum
        mu+=[minima[ii]]
        sess.run(MAP_reset,feed_dict={MAP_ini:minima[ii]})
        sigma+=[sess.run(update_TriL,feed_dict={input_data: x, sigma_corr:noise, mask: mymask})]

        # correct weighting of different minima according to El20 procedure, with samples at the maxima and well seperated maxima
        logdet  = sess.run(tf.linalg.logdet(approx_posterior_laplace.covariance()),feed_dict={input_data: x, sigma_corr:noise, mask: mymask})
        logprob = sess.run(nlPost_MAP,feed_dict={input_data: x, sigma_corr:noise, mask: mymask})
        w+=[np.exp(0.5*logdet+logprob+offset)]

    print('weights of Gaussian mixtures:', w/np.sum(w))
    mu     = np.reshape(np.asarray(mu),[1,num_comp,hidden_size])
    sigma  = np.reshape(np.asarray(sigma),[1,num_comp,hidden_size,hidden_size])
    w      = np.squeeze(np.asarray(w))

    return mu, sigma, w

def plot_prob_2D_GMM(samples, indices):
    '''
    makes corner plots of the posterior samples
    '''

    samples = samples[:,0,:]

    samples = np.hstack((np.expand_dims(samples[:,indices[0]],-1),np.expand_dims(samples[:,indices[1]],-1)))

    figure=corner.corner(samples)
    axes = np.array(figure.axes).reshape((2, 2))

    axes[1,0].set_xlabel('latent space variable %d'%indices[0])
    axes[1,0].set_ylabel('latent space variable %d'%indices[1])
    plt.savefig(plot_path+'posterior_contour_GMM_%s_latent_space_dir_%d_%d.png'%(label,indices[0],indices[1]),bbox_inches='tight')
    plt.show()
    
  

We draw 10 samples from the deepest minimum using the Laplace approximation.

In [ ]:
samples = get_laplace_sample(10,minima[0],data,custom_mask,noise,sess)

Plot the samples (with and without mask) using a pre-defined function 'plot_samples'

In [ ]:
print('samples from Laplace approximation around deepest minimum (EL2O)')
plot_samples(samples, custom_mask, title='Samples from Laplace approximation', filename='samples_laplace_deepest_minimum_%s'%label)  

Or we can use a mixture of multivariate Gaussians (GMM) as posterior approximation. Under the assumption of well separated mixture components, the EL$_2$O procedure for a GMM involves the following 3 steps: 1) finding all relevant minima in the posterior by optimization, 2) fitting Gaussians around these minima by using the Laplace approximation, 3) improving the probability distribution beyond the Gaussian approximation, if needed, 4) finding the relative weights of these mixture components. See the above pre-defined functions for referenes.

Remember that you had 15 different minimization results. How many minima points did it have? 
![alt text](pic3.png "Title")
In the above plot, you can clearly see three separate minima: iteration 0-1, 2-7, 8-9. Then, you can choose three different minima: minima[0], minima[2], minima[8]. Now, with such minima identified, we fit Gaussians around them and determine their relative weights. Number of Gaussian components we are going to use is determined by `num_comp` variable you set earlier.

In [ ]:
# here, the separate minima have to be set by hand. Take a look at your optimization results and choose three 
#different minima.
# in this run you can choose the following three different minima: minima[0],minima[4],minima[14]
mu_, sigma_, w_ = get_gmm_parameters([minima[0],minima[4],minima[14]], data, noise, custom_mask, min_loss[0])
_ = sess.run([update_w, update_mugmm,update_TriLgmm], feed_dict={mu_ini:mu_, w_ini:w_, sigma_ini:sigma_ })

This is a simple unimodal posterior example, so you should find that the first minima dominates, and the other two are negligible.

Finally, take 10 samples from GMM and plot them.

In [ ]:
samples = get_gmm_sample(10,data,custom_mask,noise,sess)
plot_samples(samples, custom_mask, title='GMM samples', filename='gmm_samples_%s'%label)  

Now, take 10000 from GMM and plot the posterior in the latent variable space using the corner package. Plot the posterior between latent variable 0 and 1, 1 and 2, and 3 and 8.

In [ ]:
more_samples = []
for ii in range(10000):
    more_samples+=[sess.run(gmm_sample,feed_dict={input_data: data, sigma_corr:noise})]
more_samples=np.asarray(more_samples)

for indices in [[0,1],[1,2],[3,8]]:
    plot_prob_2D_GMM(more_samples, indices)

Finally, using a pre-defined function 'probe_posterior', we make a comparison of posterior fits obtained with different fitting procedures to the probed true posterior. We show 3 randomly chosen latent space directions of the 10 dimensional latent space.

In [ ]:
probe_posterior(minima[0], data, noise, custom_mask, sess)

You should be able to observe that EL$_2$O gives high quality posteriors in latent space and in data space!

<span style="color:blue"> <i> 1. Using the corrupted data shown below (a broad mask is applied such thata different digits like 4s and 9s are compatible with the data - so you will be able to see a multimodal posterior!), perform the posterior analysis. </i></span> <br>
    
<span style="color:blue"> <i>(1) When you run optimization to find all local minima of the log posterior, use the following learning rate/number of iteration: 1e-1 with 10000 iterations, 1e-2 with 5000 iterations, 1e-3 with 3000 iterations. (it will take quite a bit of time to run this!) Do 10 runs. Make sure to save your results so that you don't need to run this again.</i></span> <br>

<span style="color:blue"> <i>(2) Run the full rank SVI with learning rate of 1e-3 with 3500 iterations, 1e-4 with 500 iterations, and 1e-5 with 500 iterations. Skip the mean field SVI. Make a comparison of the negative log posterior at the first two minimum with EL$_2$O vs. full rank SVI. </i></span> <br>

<span style="color:blue"> <i>(3) Fit for 5 different GMM components (corresponding to 5 different minima). Show that the first two dominates and the other three have only negligible contribution.</i></span> <br>
    

<span style="color:red"> <i> NOTE: If the kernel dies when running SVI due to the memory overflow, you can simply re-load optimization results you saved and start from there.  </i></span> <br>

In [ ]:
data_dim    = 28*28
data_size   = 1
sigma_n     = 0.1
hidden_size = 10
n_channels  = 1
seed        = 777
sample_size = 512

# settings for reconstrcution with rectangular mask
corr_type   = 'mask'
num_mnist   = 6
label       = 'solidmask'
noise_level = 0.0

In [ ]:
num_comp = ..

In [ ]:
# create underlying truth
truth = x_test[num_mnist:num_mnist+data_size]
# create corrupted data
data, custom_mask = make_corrupted_data(truth, corr_type=corr_type)
# noise
noise = get_custom_noise(data.shape, signal_dependent=False, signal=truth)

# Make plot
print('Truth:')
plot_image(truth, directory=plot_path, filename='truth_%s'%label, title='truth')
print('Data:')
plot_image(data, directory=plot_path, filename='input_data_%s'%label, title='data')

In [ ]:
def probe_posterior(minimum, x, noise, mymask, my_sess, filename=label):
    """
    make comparison: EL2O vs. full rank VI
    probes the real posterior along each latent space direction around the minimum and compares to posterior fits
    """

    _ = my_sess.run(MAP_reset,feed_dict={input_data: x, MAP_ini:minimum, sigma_corr:noise})
    _ = my_sess.run(update_mu,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    _ = my_sess.run(update_TriL,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})

    exact_hessian = sess.run(hessian,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    approx_hessian= sess.run(GN_hessian,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
    ll0 = sess.run(loss_MAP,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})


    mean_fr, h_fr, ll0_fr   = sess.run([mu_vi,hess_fr, nlPost_vi],feed_dict={input_data: x, mask:mymask, sigma_corr:noise})

    plt.figure(figsize=(4*3,3))


    jj =0
    for nn in [5,6,7]:#np.arange(hidden_size):
        H    = exact_hessian[0,nn,nn]
        Hfr  = h_fr[0,nn,nn]

        losses=[]

        subplot(1,3,jj+1)

        title('latent space direction %d'%nn)

        Delta   = 0.5
        steps   = 2000
        steps_fine = 10000
        delta_z = np.zeros((steps,hidden_size))

        delta_z[:,nn] = (np.arange(steps)-steps//2)*Delta/steps
        new_ini       = delta_z+minimum
        new_ini_fr    = delta_z+mean_fr

        full_span     = np.ones((steps_fine,hidden_size))*minimum
        full_span_min = min(min(new_ini_fr[:,nn]),min(new_ini[:,nn]),min(new_ini_fr[:,nn]))
        full_span_max = max(max(new_ini_fr[:,nn]),max(new_ini[:,nn]),max(new_ini_fr[:,nn]))
        full_span_    = np.linspace(full_span_min,full_span_max,steps_fine)
        full_span[:,nn]= full_span_

        for ii in range(steps_fine):
            _ = sess.run(MAP_reset,feed_dict={input_data: x, mask:mymask, MAP_ini:np.expand_dims(full_span[ii],axis=0), sigma_corr:noise})
            ll = sess.run(loss_MAP,feed_dict={input_data: x, mask:mymask, sigma_corr:noise})
            losses.append(ll)


        plt.plot(full_span[:,nn],losses,label='probed posterior', color='crimson',lw=2)
        plt.plot(new_ini[:,nn],ll0+H*delta_z[:,nn]**2,label='EL2O estimate (full rank)',lw=2)
        plt.plot(new_ini_fr[:,nn],ll0_fr+Hfr*delta_z[:,nn]**2,label='full rank SVI estimate',lw=2)

        plt.xlabel('z',fontsize=16)
        plt.ylim(min(min(losses),ll0_fr),max(max(losses),ll0_fr+0.5))
        if jj==0:
            plt.ylabel('negative log posterior',fontsize=14)
            plt.legend(loc='upper right',fontsize=12,framealpha=0.9)
        jj+=1
    plt.tight_layout()




    plt.savefig(plot_path+'probing_posterior_%s.pdf'%(filename),bbox_inches='tight')
    plt.show()

    

In [ ]:
...

***

#### Problem 2 - Back to Quasar (Continued from HW7 - Problem 2 - Part 7)



In [1]:
import numpy as np
from scipy.integrate import quad
#For plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load data
wavelength = np.loadtxt("HW5_Problem2_wavelength_300.txt")
X = np.loadtxt("HW5_Problem2_QSOspectra_300.txt")
ivar = np.loadtxt("HW5_Problem2_ivar_flux_300.txt")

<br><br>
The following analysis is based on https://arxiv.org/pdf/1605.04460.pdf.
<br><br>
In HW7, we reconstruct the QSO spectra from the noisy data. This reconstructed spectra is closer to the true spectra of QSO. Note that in reality, the true spectra can never be directly observed, both due to measurement error and due to absorption by intervening matter along the line of sight. So we wish to perform inference about the true spectra of QSO using a non-parametric technique called <b>Gaussian processes (GP)</b>. We henceforth call the measured spectra as $y(\lambda)$ and the true spectra as $f(\lambda)$ (where $\lambda$ refers to wavelength).
<br><br>
A gaussian process is fully specified by its first two central moments: a mean function $\mu(\lambda)$ and a covariance function $K(\lambda, \lambda')$: <br><br>
$$ \mu(\lambda) = \mathbb{E}[f(\lambda)\ \vert\ \lambda] $$<br>
$$ K(\lambda, \lambda') = \mathrm{cov}[f(\lambda), f(\lambda')\ \vert\ \lambda, \lambda'] $$.
<br>
In this problem, we can derive the posterior distribution of $f$ conditioned on the observed values of $y$: <br><br>
$$ p(f^*\ \vert\ \lambda^*, \lambda, y, \sigma(\lambda)^2) = \mathcal{N}(f^*\ \vert\ \mu_{f|y}(\lambda^*), K_{f|y}(\lambda^*, \lambda^{*,})) $$
<br>
where $\mathcal{N}(f\ \vert\ \mu, K)$ is a multivariate Gaussian given by: <br>
$$ \mathcal{N}(f\ \vert\ \mu, K) = \frac{1}{\sqrt{(2\pi)^d \mathrm{det}K}} \mathrm{exp}\big(-\frac{1}{2}(f-\mu)^TK^{-1}(f-\mu)\big) $$<br>
where $d$ is the dimension of $f$.
<br><br><br><br>
In other words, for the QSO $i$, the measured spectra $y$ is $X_{row\ i}$. Then, we can compute the posterior distribution of $f$ given $X_{row\ i}$ as:
<br><br>
$$ \mu + \mathcal{N}\big(f\ \big\vert\ \mu_{f|X_{row\ i}}, K_{f|X_{row\ i}}\big) $$
<br>
where $\mu$ is given by:
<br><br>
$ \mu$ $ =
    \begin{bmatrix}
        \overline{x}_1 & \overline{x}_2 & \dots  &  \overline{x}_{824} \\
    \end{bmatrix}.$
<br><br>
The mean function $\mu_{f|X_{row\ i}}$ and the covariance function $K_{f|X_{row\ i}}$ are defined as:<br><br>
$$ \mu_{f|X_{row\ i}} = \mu\ +\ K(K + V)^{-1}(X_{row\ i} - \mu)  $$<br>
$$ K_{f|X_{row\ i}} = K - K(K + V)^{-1}K$$
<br>
where $K = \phi \phi^T$ (We can use $\phi$ from Part 7, Problem 2, HW7. $\phi$ is a matrix of eigenvectors, its dimension is "nLambda" x "nEigvec"). <br><br>$V$ is a diagonal matrix whose entries are $\sigma(\lambda)^2$ e.g. for the QSO $i$, $V$ = np.diag(1/ivar[i,:]).
<br><br>
Finally, we can plot $f(\lambda)$ by sampling from $\mathcal{N}\big(f\ \big\vert\ \mu_{f|X_{row\ i}}, K_{f|X_{row\ i}}\big)$.
<br><br>
<span style="color:blue"> <i> 1. For any two spectra, plot $f(\lambda)$ using Gaussian processes. You can use np.random.multivariate_normal (https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.multivariate_normal.html) to sample from a multivariate Gaussian. </i></span> <br>

In [ ]:
...

***

## To Submit
Execute the following cell to submit.
If you make changes, execute the cell again to resubmit the final copy of the notebook, they do not get updated automatically.<br>
__We recommend that all the above cells should be executed (their output visible) in the notebook at the time of submission.__ <br>
Only the final submission before the deadline will be graded. 


In [ ]:
_ = ok.submit()